In [1]:
import numpy as np
from chesslab_.utils import params,load_pkl
from chesslab_.training_tf import data_loader
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
np.random.seed(0)
tf.random.set_seed(0)

batch_size = params.batch_size
path = 'D:/database/ccrl/'

name=f'{path}ccrl_states_elo2.pkl'
x_data = load_pkl(name)

name=f'{path}ccrl_results_elo2.pkl'
y_data = load_pkl(name)[:,1]

print(x_data.shape)

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size = params.test_size, random_state = 0, shuffle = True)

del x_data
del y_data


save_name='./tmp/tf_elo.0'

(6291280, 64)


In [3]:
lr = 0.1
mo = 0.1
start=0
epochs=30
# Instantiate model
print(lr)
encoding_1={
    '.':np.array([0,0,0],dtype=np.float),
    'p':np.array([0,0,1],dtype=np.float),
    'P':np.array([0,0,-1],dtype=np.float),
    'b':np.array([0,1,0],dtype=np.float),
    'B':np.array([0,-1,0],dtype=np.float),
    'n':np.array([1,0,0],dtype=np.float),
    'N':np.array([-1,0,0],dtype=np.float),
    'r':np.array([0,1,1],dtype=np.float),
    'R':np.array([0,-1,-1],dtype=np.float),
    'q':np.array([1,0,1],dtype=np.float),
    'Q':np.array([-1,0,-1],dtype=np.float),
    'k':np.array([1,1,0],dtype=np.float),
    'K':np.array([-1,-1,0],dtype=np.float)
}

encoding_2={
    '.':np.array([0,0,0,0],dtype=np.float),
    'p':np.array([1,0,0,0],dtype=np.float),
    'P':np.array([0,0,0,1],dtype=np.float),
    'b':np.array([0,1,0,0],dtype=np.float),
    'B':np.array([0,0,1,0],dtype=np.float),
    'n':np.array([1,1,0,0],dtype=np.float),
    'N':np.array([0,0,1,1],dtype=np.float),
    'r':np.array([1,0,1,0],dtype=np.float),
    'R':np.array([0,1,0,1],dtype=np.float),
    'q':np.array([1,0,0,1],dtype=np.float),
    'Q':np.array([0,1,1,0],dtype=np.float),
    'k':np.array([1,1,1,0],dtype=np.float),
    'K':np.array([0,1,1,1],dtype=np.float)
}


encoding=encoding_1
for i in encoding.items(): print(i)

inter_encoding = {params.inter_map[i]:code for i,code in encoding.items()}
print(inter_encoding)

0.1
('.', array([0., 0., 0.]))
('p', array([0., 0., 1.]))
('P', array([ 0.,  0., -1.]))
('b', array([0., 1., 0.]))
('B', array([ 0., -1.,  0.]))
('n', array([1., 0., 0.]))
('N', array([-1.,  0.,  0.]))
('r', array([0., 1., 1.]))
('R', array([ 0., -1., -1.]))
('q', array([1., 0., 1.]))
('Q', array([-1.,  0., -1.]))
('k', array([1., 1., 0.]))
('K', array([-1., -1.,  0.]))
{0: array([0., 0., 0.]), 1: array([0., 0., 1.]), 2: array([ 0.,  0., -1.]), 3: array([0., 1., 0.]), 4: array([ 0., -1.,  0.]), 5: array([1., 0., 0.]), 6: array([-1.,  0.,  0.]), 7: array([0., 1., 1.]), 8: array([ 0., -1., -1.]), 9: array([1., 0., 1.]), 10: array([-1.,  0., -1.]), 11: array([1., 1., 0.]), 12: array([-1., -1.,  0.])}


In [4]:
model = tf.keras.Sequential([
      tf.keras.layers.InputLayer(input_shape=( 8, 8, 3)), 
      tf.keras.layers.Conv2D(32, 7, activation='elu', padding='same'),
      tf.keras.layers.Conv2D(64, 5, activation='elu', padding='same'),
      tf.keras.layers.Conv2D(128, 3, activation='elu', padding='same'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(256, activation='elu'),
      tf.keras.layers.Dense(2, activation='softmax')])
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=tf.keras.metrics.SparseCategoricalAccuracy()
)

In [ ]:
class MyCustomCallback(tf.keras.callbacks.Callback):
    def __init__(self,path):
        super(MyCustomCallback, self).__init__()
        self.path=path
  
    def on_epoch_end(self, epoch, logs=None):
        name = '{}.{}.h5'.format(self.path,epoch)
        self.model.save(name) #guarda el modelo

cp_callback = MyCustomCallback(checkpoint_path)

In [5]:


#loss_fn = training.loss_fn
#optim = training.optim

#training.set_encoding(encoding)

train_loader=data_loader( x_data = x_train,y_data=y_train,batch_size=batch_size,shuffle=True ,encoding = encoding,callbacks=[cp_callback] )
test_loader=data_loader( x_data = x_test,y_data=y_test,batch_size=batch_size,shuffle=False , encoding = encoding )

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 8, 8, 32)          4736      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 2,227,778
Trainable params: 2,227,778
Non-trainable params: 0
______________________________________________

In [6]:
for x,y in train_loader:
    print(x.shape, y.shape)
    #model(x.to(device))
    break

(128, 8, 8, 3) (128,)


In [7]:
inter_encoding

{0: array([0., 0., 0.]),
 1: array([0., 0., 1.]),
 2: array([ 0.,  0., -1.]),
 3: array([0., 1., 0.]),
 4: array([ 0., -1.,  0.]),
 5: array([1., 0., 0.]),
 6: array([-1.,  0.,  0.]),
 7: array([0., 1., 1.]),
 8: array([ 0., -1., -1.]),
 9: array([1., 0., 1.]),
 10: array([-1.,  0., -1.]),
 11: array([1., 1., 0.]),
 12: array([-1., -1.,  0.])}

In [8]:
history = model.fit(train_loader,verbose=1,epochs=epochs,validation_data=test_loader)

Epoch 1/30
44236/44236 [==============================] - 321s 7ms/step - loss: 0.5428 - sparse_categorical_accuracy: 0.7103 - val_loss: 0.5095 - val_sparse_categorical_accuracy: 0.7353
Epoch 2/30
22715/44236 [==============>...............] - ETA: 2:12 - loss: 0.5068 - sparse_categorical_accuracy: 0.7371

KeyboardInterrupt: 